## Loading ROOT libraries

In [2]:
from ROOT import TCanvas, TPad, TFile, TPaveLabel, TPaveText
from ROOT import gROOT
import root_numpy
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (LinearLocator, MultipleLocator, FormatStrFormatter,AutoMinorLocator)
%jsroot on
#ROOT.gStyle.SetOptStat(0)
#ROOT.gStyle.SetOptStat(1111111)

In [6]:
run = [2163]
tip = "3D"

filename = "../reco_files/6040/new_reco_run%05d_%s.root" % (run[0],tip)
tf = ROOT.TFile(filename)
tree = tf.Get('Events')

In [34]:
## Plot parameters
caption = 0
logy = 0
logx = 0
stats = 1
colors = [ROOT.kGray+1]

In [45]:
if (c):
    c.Close()
    ROOT.gSystem.ProcessEvents()
    c = 0

var = "cl_integral"
hist1 = 'Integral'
it=2

ROOT.gStyle.SetOptStat(0)

c = ROOT.TCanvas('','',800,600)

#-------2D Method------#
#it=1
h = ROOT.TH1F('{hist1}'.format(hist1=hist1),'',3200,-800,5400)
h.Sumw2()
tree.Draw("{vr}*{fac}>>{hist1}".format(fac=fac,vr=var,hist1=hist1),"TMath::Hypot(cl_xmean-1024,(cl_ymean-1024)*1.2)<400 && cl_iteration==2 && cl_nhits>30")
h.SetMarkerStyle(20)
h.SetMarkerSize(0.5)
h.SetMarkerColor(colors[0])
h.SetFillColor(ROOT.kGray)
h.SetLineColor(ROOT.kGray+1)
h.Rebin(10)
h.Rebin(4)
#h.SetFillStyle(3005)

h.Draw('hist')
ROOT.gPad.Update()

#h.GetYaxis().SetRangeUser(0,y_lim[i])
h.GetXaxis().SetRangeUser(0,4400)
h.GetXaxis().SetTitle("Cluster Light [photons]")
h.GetYaxis().SetTitle("Number of Clusters")

## FIT Paramters and PLOT
myPolya = ROOT.TF1("myPolya","[0]*ROOT::Math::negative_binomial_pdf(x,[1],[2])",0,10000);
myExp = ROOT.TF1("myExp","[0]*exp(-x/[1])",400,10000);
myPolyaE = ROOT.TF1("myPolyaE","[0]*ROOT::Math::negative_binomial_pdf(x,[1],[2])+[3]*exp(-x/[4])",0,10000);

myPolyaE.SetParameters(10000,0.4,2400,1e2,1e3)
myPolyaE.SetLineColor(ROOT.kGray+2)

h.Fit("myPolyaE","","same",500,4400)

myPolya.SetParameters(myPolyaE.GetParameter(0),myPolyaE.GetParameter(1),myPolyaE.GetParameter(2))
myPolya.SetLineColor(ROOT.kBlack)
myPolya.Draw("same")

#-------Stat Box --------#
dif = 0.28
ypos = 0.35

if stats:
    rMean = ((1-myPolya.GetParameter(1))*myPolya.GetParameter(2))/myPolya.GetParameter(1)
    rSigma = ROOT.TMath.Sqrt((((1-myPolya.GetParameter(1))*myPolya.GetParameter(2))/(myPolya.GetParameter(1)*myPolya.GetParameter(1))))
    
    box = ROOT.TBox(2800,2*63.7,4425,2*80.2);
    box.SetFillStyle(0);
    box.SetLineColor(1);
    box.SetLineWidth(1);
    box.Draw("same");

    latex = ROOT.TLatex()
    latex.SetTextSize(0.04);
    latex.SetTextAlign(13);

    latex.DrawLatex(2900, 2*77.9,ROOT.Form("#bf{Mean                  %.0f}" % rMean));
    latex.DrawLatex(2900, 2*73.4,ROOT.Form("#bf{Sigma                 %.0f}" % rSigma));


if caption:
    #-------Legend------#
    difx = 0.245
    xpos = 0.5
    (x1,y1,x2,y2) = (xpos, .75, xpos+difx, .89)
    leg = ROOT.TLegend(x1,y1,x2,y2)
    leg.AddEntry(h,'{hist1}'.format(hist1=hist1),'f')
    leg.Draw()
if logx:
    c.SetLogx()
if logy:
    c.SetLogy()
    
c.SetGrid()
c.Draw()
#c.Close()

 FCN=117.858 FROM MIGRAD    STATUS=CONVERGED    1269 CALLS        1270 TOTAL
                     EDM=7.1464e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           4.63948e+04   1.93092e+03   9.69421e+00   1.15763e-07
   2  p1           1.84812e-02   1.55217e-03   5.64275e-07  -5.08601e+00
   3  p2           5.69931e+01   4.83702e+00   1.75755e-03   1.46470e-03
   4  p3           2.63094e+02   1.24583e+02   1.04017e-01  -2.28093e-05
   5  p4           2.99864e+02   5.19746e+01   4.37435e-02  -4.38718e-05


Warning in <TFile::Append>: Replacing existing TH1: Integral (Potential memory leak).


In [ ]:
latex.SetTextSize()